In [1]:
#default_exp qdrant

In [2]:
#export
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

from qdrant_client import QdrantClient
collection_name = "food"

docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    --name qdrant \
    qdrant/qdrant
docker update --restart unless-stopped qdrant

In [3]:
#export
client = QdrantClient(host='localhost', port=6333)
dim = 768 #+ onehot.n_dim()
client.get_collections = lambda: [c['name'] for c in client.http.collections_api.get_collections().dict()['result']['collections']]
client.collection_len = lambda name: client.http.collections_api.get_collection(name).dict()['result']['vectors_count']


In [12]:
collection_name = "fundation_food"

In [13]:
# client.recreate_collection(collection_name,dim)

In [16]:
client.get_collections()

['fundation_food', 'glovo', 'food']

In [15]:
client.collection_len(collection_name)

0

In [9]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_classfy_image.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted inference.ipynb.
Converted stats.ipynb.
Converted usda_parsing.ipynb.
